<a href = 'https://dhds.cdc.gov/CR?ReportType=Years&LocationId=59&StratCatId=RACE&YearId=YR5&CompareColumns=YR5&ShowCategoryIds=PREVENT,BARRIER&SearchString=&ShowReportOptions=false&CategoryFilterId=&StratFilterId=&ShowCI=true&ShowWN=true&HideMissingDataRows=false'>Data Source</a>

In [2]:
import pandas as pd
import numpy as np 
import os
import altair as alt

In [3]:
df = pd.read_csv("../data/preventitive_care.csv")

<h2>Manipulate DataFrame</h2>

In [4]:
df = df.drop(columns = ['FootnoteText', 'Url', 'FootnoteType'])

In [5]:
preventitive_care = df[(df.Category == 'Prevention & Screenings') & (df.Year == 2020.0)
                       & (df.Response == 'Yes')]

In [6]:
preventitive_care[preventitive_care.Stratification1 != 'Any Disability'][['Stratification1', 'Question', 'WeightedNumber']]



,Stratification1,Question,WeightedNumber
1741,Cognitive Disability,Mammogram in the past 2 years among females 50...,3631869.0
1744,Hearing Disability,Mammogram in the past 2 years among females 50...,2098659.0
1747,Mobility Disability,Mammogram in the past 2 years among females 50...,7159511.0
1750,Vision Disability,Mammogram in the past 2 years among females 50...,2128751.0
1753,No Disability,Mammogram in the past 2 years among females 50...,25754412.0
1777,Cognitive Disability,Up-to-date cervical cancer screening among fem...,6297386.0
1780,Hearing Disability,Up-to-date cervical cancer screening among fem...,1422602.0
1783,Mobility Disability,Up-to-date cervical cancer screening among fem...,4660307.0
1786,Vision Disability,Up-to-date cervical cancer screening among fem...,2353139.0
1789,No Disability,Up-to-date cervical cancer screening among fem...,47502681.0


In [7]:
preventitive_care['Data_Value'] = preventitive_care.Data_Value / 100
preventitive_care['High_Confidence_Limit'] = preventitive_care.High_Confidence_Limit / 100
preventitive_care['Low_Confidence_Limit'] = preventitive_care.Low_Confidence_Limit / 100

/var/folders/49/3f_15lzd6b57lnzlk4m_9k440000gn/T/ipykernel_828/2997796587.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preventitive_care['Data_Value'] = preventitive_care.Data_Value / 100
/var/folders/49/3f_15lzd6b57lnzlk4m_9k440000gn/T/ipykernel_828/2997796587.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preventitive_care['High_Confidence_Limit'] = preventitive_care.High_Confidence_Limit / 100
/var/folders/49/3f_15lzd6b57lnzlk4m_9k440000gn/T/ipykernel_828/2997796587.py:3: SettingWithCopyWarni

In [8]:
disabled_mean = pd.DataFrame(preventitive_care[preventitive_care.Stratification1 == 'Any Disability']\
                             .groupby('Question')['Data_Value'].mean())

disabled_mean = disabled_mean.reset_index()

In [9]:
vals = ['Data_Value', 'High_Confidence_Limit', 'Low_Confidence_Limit']
mean_by_type = preventitive_care[preventitive_care.Stratification1 != 'Any Disability']\
                .groupby(['Stratification1', 'Question'])[vals].mean()
mean_by_type = mean_by_type.reset_index()

In [10]:
mean_by_type = mean_by_type.merge(disabled_mean, on = 'Question', suffixes = ('', '_Disabled_Mean'))
mean_by_type['delta'] = mean_by_type['Data_Value'] - mean_by_type['Data_Value_Disabled_Mean']

mean_by_type['upper_val_delta'] = mean_by_type['High_Confidence_Limit'] - mean_by_type['Data_Value_Disabled_Mean']
mean_by_type['lower_val_delta'] = mean_by_type['Low_Confidence_Limit'] - mean_by_type['Data_Value_Disabled_Mean']

#mean_by_type['delta'] = (mean_by_type['Data_Value'] - mean_by_type['Data_Value_Disabled_Mean']) / mean_by_type['Data_Value_Disabled_Mean']

#mean_by_type['upper_val_delta'] = (mean_by_type['High_Confidence_Limit'] - mean_by_type['Data_Value_Disabled_Mean']) / mean_by_type['Data_Value_Disabled_Mean']
#mean_by_type['lower_val_delta'] = (mean_by_type['Low_Confidence_Limit'] - mean_by_type['Data_Value_Disabled_Mean']) /  mean_by_type['Data_Value_Disabled_Mean']

In [11]:
 mean_by_type['radio_option'] = mean_by_type.Stratification1.apply(lambda s: np.where(s == 'No Disability', 
                                                                                      'Include Non-Disabled People', 
                                                                                      'Exclude Non-Disabled People'))

In [12]:
mean_by_type['margin_of_error'] = mean_by_type['High_Confidence_Limit'] - mean_by_type['Data_Value']

In [13]:
mean_by_type

,Stratification1,Question,Data_Value,High_Confidence_Limit,Low_Confidence_Limit,Data_Value_Disabled_Mean,delta,upper_val_delta,lower_val_delta,radio_option,margin_of_error
0,Cognitive Disability,Had a flu vaccine in the past 12 months among ...,0.413,0.424,0.402,0.418,-0.005,0.006,-0.016,Exclude Non-Disabled People,0.011
1,Hearing Disability,Had a flu vaccine in the past 12 months among ...,0.407,0.425,0.389,0.418,-0.011,0.007,-0.029,Exclude Non-Disabled People,0.018
2,Mobility Disability,Had a flu vaccine in the past 12 months among ...,0.441,0.458,0.424,0.418,0.023,0.040,0.006,Exclude Non-Disabled People,0.017
3,No Disability,Had a flu vaccine in the past 12 months among ...,0.449,0.453,0.444,0.418,0.031,0.035,0.026,Include Non-Disabled People,0.004
4,Vision Disability,Had a flu vaccine in the past 12 months among ...,0.383,0.401,0.364,0.418,-0.035,-0.017,-0.054,Exclude Non-Disabled People,0.018
5,Cognitive Disability,Mammogram in the past 2 years among females 50...,0.722,0.741,0.702,0.735,-0.013,0.006,-0.033,Exclude Non-Disabled People,0.019
6,Hearing Disability,Mammogram in the past 2 years among females 50...,0.752,0.776,0.726,0.735,0.017,0.041,-0.009,Exclude Non-Disabled People,0.024
7,Mobility Disability,Mammogram in the past 2 years among females 50...,0.740,0.755,0.724,0.735,0.005,0.020,-0.011,Exclude Non-Disabled People,0.015
8,No Disability,Mammogram in the past 2 years among females 50...,0.804,0.812,0.797,0.735,0.069,0.077,0.062,Include Non-Disabled People,0.008
9,Vision Disability,Mammogram in the past 2 years among females 50...,0.701,0.735,0.664,0.735,-0.034,0.000,-0.071,Exclude Non-Disabled People,0.034


Generate lower and upper bounds for scale parameter

In [14]:
t = mean_by_type.groupby('Question')[['lower_val_delta', 'upper_val_delta']].agg(func = [max, min])\
.apply(lambda s: abs(s))
t.columns = t.columns.droplevel(0)

In [15]:
t = t.apply(lambda s: max(s), axis = 1)
t = pd.DataFrame(data = {'q_max_moe': t})

In [16]:
t['q_min_moe'] = t.q_max_moe.apply(lambda s: s * -1)
t = t.reset_index()

In [17]:
mean_by_type = mean_by_type.merge(t, on = 'Question')

In [18]:
mean_by_type['annotation_indicator'] = mean_by_type['Data_Value_Disabled_Mean'] - mean_by_type['Data_Value_Disabled_Mean']

In [19]:
mean_by_type['disabled_binary'] = mean_by_type['Stratification1'].apply(lambda s: s == 'No Disability')

In [20]:
questions = list(mean_by_type.Question.unique())
zero_df = pd.DataFrame({'x': [0]})
sorting = ['Cognitive Disability', 'Hearing Disability', 'Mobility Disability', 
           'Vision Disability', 'No Disability']
radio_options = ['Include Non-Disabled People', 'Exclude Non-Disabled People']

select_care_type = alt.selection_single(
    fields = ['Question'], 
    init = {'Question' : questions[0]}, 
    bind = alt.binding_select(options = questions, name = 'Select Preventive Care Type: '), 
    on = 'dragover', 
    clear = 'false'
)

# include_non_disabled_radio = alt.binding_radio(options = radio_options)
# non_disabled_select = alt.selection_single(fields = ['radio_option'], 
#                                            bind = include_non_disabled_radio, 
#                                            name = "Rating")

input_checkbox = alt.binding_checkbox()
checkbox_selection = alt.selection_single(bind= input_checkbox, name = "Ability Types")

# size_checkbox_condition = alt.condition(checkbox_selection,
#                                         alt.SizeValue(25),
#                                         alt.Size('Hundred_Million_Production:Q')
#                                        )


lines = alt.Chart(mean_by_type).mark_rule(
    color = 'blue', 
    size = 3
).encode(
    alt.Y('Stratification1:N', 
          title = 'Type of Disability', sort = sorting), 
    alt.X('upper_val_delta:Q', 
          title = 'Difference from Average For All Disabilities (Percentage Points)',
          scale = alt.Scale(domainMid = 0)),
    alt.X2('lower_val_delta:Q'),
    color = alt.Color('disabled_binary:N'),
    tooltip = [alt.Tooltip('Data_Value:Q', title = 'Percent Up To Date', format = '.1%'), 
               alt.Tooltip('margin_of_error:Q', title = 'Margin of Error', format = '.1%')
               #alt.Tooltip('High_Confidence_Limit:Q', title = 'High Estimate', format = '.2%'), 
               #alt.Tooltip('Low_Confidence_Limit:Q', title = 'Low Estimate', format = '.2%')
              ]
).transform_filter(select_care_type)

mean = alt.Chart(mean_by_type).add_selection(select_care_type).mark_point(
    size = 50
).encode(
    y = alt.Y('Stratification1:N', sort = sorting), 
    x = alt.X('delta:Q', 
              scale = alt.Scale(domainMid = 0)), 
    color = alt.Color('disabled_binary:N'),
    tooltip = [alt.Tooltip('Data_Value:Q', title = 'Percent Up To Date', format = '.1%'), 
               alt.Tooltip('margin_of_error:Q', title = 'Margin of Error', format = '.1%')
               #alt.Tooltip('High_Confidence_Limit:Q', title = 'High Estimate', format = '.2%'), 
               #alt.Tooltip('Low_Confidence_Limit:Q', title = 'Low Estimate', format = '.2%')
              ]
).transform_filter(select_care_type)

zero = alt.Chart(zero_df).mark_rule(
    color = 'lightblue', 
    size = 2.5
).encode(
    x = alt.X('x')
)

mean_annotation_num = alt.Chart(mean_by_type).mark_text(
    fontSize = 9.5, 
    dx = 135, 
    dy = 140, 
    align = 'left'
).encode(
    x = alt.X('annotation_indicator:Q'), 
    text = alt.Text('Data_Value_Disabled_Mean', format = '.0%')
).transform_filter(select_care_type)

mean_annotation_text = alt.Chart(mean_by_type).mark_text(
    fontSize = 9.5, 
    dx = 5, 
    dy = 140, 
    align = 'left',
    text = 'Average For Disabled People:'
).encode(
    x = alt.X('annotation_indicator:Q')
)

alt.layer(zero, lines, mean, mean_annotation_num, mean_annotation_text
).properties(
    width = 600, 
    height = 300
)

alt.LayerChart(...)

With Raw Values

In [248]:
questions = list(mean_by_type.Question.unique())
zero_df = pd.DataFrame({'x': [0]})
sorting = ['Cognitive Disability', 'Hearing Disability', 'Mobility Disability', 
           'Vision Disability', 'No Disability']
radio_options = ['Include Non-Disabled People', 'Exclude Non-Disabled People']

select_care_type = alt.selection_single(
    fields = ['Question'], 
    init = {'Question' : questions[0]}, 
    bind = alt.binding_select(options = questions, name = 'Select Preventive Care Type: '), 
    on = 'dragover', 
    clear = 'false'
)

input_checkbox = alt.binding_checkbox()
checkbox_selection = alt.selection_single(bind= input_checkbox, name = "Ability Types")


lines = alt.Chart(mean_by_type).mark_rule(
    color = 'blue', 
    size = 3
).encode(
    alt.Y('Stratification1:N', 
          title = 'Type of Disability', sort = sorting), 
    alt.X('High_Confidence_Limit:Q', 
          title = 'Percent Up To Date',
          axis = alt.Axis(format = '.0%')),
    alt.X2('Low_Confidence_Limit:Q'),
    color = alt.Color('disabled_binary:N'),
    tooltip = [alt.Tooltip('Data_Value:Q', title = 'Percent Up To Date', format = '.1%'), 
               alt.Tooltip('margin_of_error:Q', title = 'Margin of Error', format = '.1%')]
).transform_filter(select_care_type)

mean = alt.Chart(mean_by_type).add_selection(select_care_type).mark_point(
    size = 50
).encode(
    y = alt.Y('Stratification1:N', sort = sorting), 
    x = alt.X('Data_Value:Q'), 
    color = alt.Color('disabled_binary:N'),
    tooltip = [alt.Tooltip('Data_Value:Q', title = 'Percent Up To Date', format = '.1%'), 
               alt.Tooltip('margin_of_error:Q', title = 'Margin of Error', format = '.1%')]
).transform_filter(select_care_type)

zero = alt.Chart(mean_by_type).mark_rule(
    color = 'lightblue', 
    size = 2.5
).encode(
    x = alt.X('Data_Value_Disabled_Mean')
).transform_filter(select_care_type)

mean_annotation_num = alt.Chart(mean_by_type).mark_text(
    fontSize = 9.5, 
    dx = 135, 
    dy = 140, 
    align = 'left'
).encode(
    x = alt.X('Data_Value_Disabled_Mean:Q'), 
    text = alt.Text('Data_Value_Disabled_Mean', format = '.0%')
).transform_filter(select_care_type)

mean_annotation_text = alt.Chart(mean_by_type).mark_text(
    fontSize = 9.5, 
    dx = 5, 
    dy = 140, 
    align = 'left',
    text = 'Average For Disabled People:'
).encode(
    x = alt.X('Data_Value_Disabled_Mean:Q')
).transform_filter(select_care_type)

alt.layer(zero, lines, mean, mean_annotation_num, mean_annotation_text
).properties(
    width = 600, 
    height = 300
)

alt.LayerChart(...)